In [ ]:
# !pip install transformers==4.2.2
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-vinryzxl
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-vinryzxl
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 596 kB 4.2 MB/s 
     |████████████████████████████████| 6.8 MB 79.6 MB/s 
     |████████████████████████████████| 895 kB 70.0 MB/s 
     |████████████████████████████████| 67 kB 6.4 MB/s 
  Created wheel for transformers: filename=transformers-4.16.0.dev0-py3-none-any.whl size=3491522 sha256=bc3d4c8a2314a63c624aa7195649a2b9137c001009d79781c4e15ccd25949804
  Stored in directory: /tmp/pip-ephem-wheel-cache-qgv8ejw5/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully unin

In [ ]:
!nvidia-smi

Thu Jan 20 01:04:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pandas as pd


from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import TextDataset,DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments,AutoModelForCausalLM
from transformers import pipeline

In [ ]:
IN_COLAB: bool = False
try:
  from google.colab import drive, files
  IN_COLAB = True
except:
  IN_COLAB = False

path: str = ''
if IN_COLAB:
  drive.mount('/content/drive')
  path = 'drive/MyDrive/RapMachine/KeywordLyrics.json'
else:
  path = None
df: pd.DataFrame = pd.read_json(path)

Mounted at /content/drive


# Funktioniert

In [ ]:
import re
import json
from sklearn.model_selection import train_test_split


def build_text_file(df: pd.DataFrame, dest_path: str) -> None:
  text: str = ''
  for _, row in df.iterrows():
    keywords: str = row[0]
    lyrics: str = row[1]
    line: str = str(lyrics.strip())
    line = re.sub(r"\s", " ", line)
    text += line + "  "
    # processed: str  = re.sub(r"\n", " ", lyrics.strip())
    # processed: str = re.sub(r'\n', '<|newline|>', lyrics)
    # prompt: str = f"<|startoftext|>CONTEXT: {keywords} RAP-LYRICS: {processed}<|endoftext|>\n"
    # text += prompt
    # text += processed
  f = open(dest_path, 'w')
  f.write(text)
  f.close()


train, test = train_test_split(df,test_size=0.15) 
build_text_file(train, './training_data/data_train.txt')
build_text_file(test, './training_data/data_test.txt')
pass

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    'gpt2')
    # bos_token='<|startoftext|>',
    # eos_token='<|endoftext|>',)
    # additional_special_tokens=['<|newline|>'])

In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_path = './training_data/data_train.txt'
test_path = './training_data/data_test.txt'
train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
model_path = './RM'

In [ ]:
model = AutoModelForCausalLM.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))


training_args = TrainingArguments(
    output_dir=model_path, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=2, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 200, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [ ]:
trainer.train()
trainer.save_model()

Step,Training Loss
500,4.318700
1000,4.011900


In [ ]:
rapper = pipeline('text-generation',model=model_path, tokenizer='gpt2',config={"num_beams":5, 'min_length': 200})

In [ ]:
input: str = '<|startoftext|>CONTEXT: fore it crash, sky falling, Drop that ass, fore, higher, falling, Drop, crash, sky, ass, baby, switched my robot RAP-LYRICS:'
r = rapper('Straight')
output: str = str(r[0]['generated_text'])
print(output)
print(len(output))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Straight outta town, I'm from Detroit And your ass smell like your dick is full of shit So I got to say I'm in love with the way she moves But that pussy is so tight, she can't hold me down I guess
197


# Test Prompt

In [ ]:
import re
import json
from sklearn.model_selection import train_test_split


def build_text_file(df: pd.DataFrame, dest_path: str) -> None:
  text: str = ''
  for _, row in df.iterrows():
    keywords: str = row[0]
    lyrics: str = row[1].strip()
    # lyrics = re.sub(r"\s", " ", lyrics)
    # lyrics= re.sub(r'\n', '<|newline|>', lyrics)
    # lyrics= re.sub(r'\n', ' ', lyrics)
    # prompt: str = f"<|startoftext|>CONTEXT: {keywords}\nRAP-LYRICS: {lyrics}<|endoftext|>\n"
    prompt: str = f"CONTEXT: {keywords}\nRAP-LYRICS: {lyrics}\n"
    text += prompt
  f = open(dest_path, 'w')
  f.write(text)
  f.close()


train, test = train_test_split(df,test_size=0.15) 
build_text_file(train, './training_data/data_train.txt')
build_text_file(test, './training_data/data_test.txt')
pass

In [ ]:
model_name: str = "hivemind/gpt-j-6B-8bit"

model_path = './RM'
train_path = './training_data/data_train.txt'
test_path = './training_data/data_test.txt'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name)
    # bos_token='<|startoftext|>',
    # eos_token='<|endoftext|>')
    # additional_special_tokens=['<|newline|>'])

model = AutoModelForCausalLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

https://huggingface.co/hivemind/gpt-j-6B-8bit/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpj_w9214u


Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

storing https://huggingface.co/hivemind/gpt-j-6B-8bit/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/1ab2dafc24948fffd9dcef597dd54769fe4be338193cf42cbf18a1dc854d031c.d95c0c78181e81d23ac523ac4c75230cc27dff013bdbe768a773586428f7b889
creating metadata file for /root/.cache/huggingface/transformers/1ab2dafc24948fffd9dcef597dd54769fe4be338193cf42cbf18a1dc854d031c.d95c0c78181e81d23ac523ac4c75230cc27dff013bdbe768a773586428f7b889
loading configuration file https://huggingface.co/hivemind/gpt-j-6B-8bit/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/1ab2dafc24948fffd9dcef597dd54769fe4be338193cf42cbf18a1dc854d031c.d95c0c78181e81d23ac523ac4c75230cc27dff013bdbe768a773586428f7b889
Model config GPTJConfig {
  "_name_or_path": "hivemind/gpt-j-6B-8bit",
  "activation_function": "gelu_new",
  "architectures": [
    "GPTJForCausalLM"
  ],
  "attn_pdrop": 0.0,
  "bos_token_id": 50256,
  "embd_pdrop": 0.0,
  "eos_token_id": 50256,
  "gradient_checkp

OSError: ignored

In [ ]:
def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator


train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Token indices sequence length is longer than the specified maximum sequence length for this model (3022817 > 2048). Running this sequence through the model will result in indexing errors


In [ ]:
training_args = TrainingArguments(
    output_dir=model_path, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=2, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 200, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [ ]:
trainer.train()
trainer.save_model()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 23615
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1476


RuntimeError: ignored

In [ ]:
rapper = pipeline(
    'text-generation',
    model=model_path,
    tokenizer='gpt2',
    config={
        "num_beams": 5,
        'min_length': 400,
        'top-k': 50})

In [ ]:
keywords: str = 'fore it crash, sky falling, Drop that ass, fore, higher, falling, Drop, crash, sky, ass, baby'
input: str = f"CONTEXT: {keywords}\nRAP-LYRICS:"
r = rapper(input)
output: str = str(r[0]['generated_text'])
output

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'CONTEXT: fore it crash, sky falling, Drop that ass, fore, higher, falling, Drop, crash, sky, ass, baby\nRAP-LYRICS: I’m back in my groove! What? What?'

# Data Doc

- Config 1
5 Epochs and substituting '\n' characters to spaces, while keeping the prompt format `CONTEXT: {keywords}\nRAP-LYRICS: {lyrics}` leads to short output. However, the quality of the output is okish.

- Config 2
bos and eos are added to the tokenizer. Input is also modified to `<|startoftext|>CONTEXT: {keywords} RAP-LYRICS: {lyrics}<|endoftext|>\n`. Training with 5 Epochs.
  - `<|newline|>` character apperars. Text is still too short.
- Config 3
Config 2 tested again, reloaded, and with 3 epochs only. Still too short

- Config 4
 `<|startoftext|>CONTEXT: {keywords} RAP-LYRICS: {lyrics}<|endoftext|>\n`. lyrics
 will be preprocessesed with `<|newline|>` character. Epchs: 3. Still too short.

- Config 5
 `<|startoftext|>CONTEXT: {keywords} RAP-LYRICS: {lyrics}<|endoftext|>\n`. lyrics
 will be preprocessesed with `<|newline|>` character but not included in tokenizer. Epchs: 3. Still too short.
 - Config 6
 Feeding ONLY plain text without newlines and no prompts. 2 Epochs, GPT2, Steps reduced to 200. min_length=200 and beam search. Works properly